<a href="https://colab.research.google.com/github/lamtung1997/Solving_PDEs_Deep_learning/blob/main/3_Stokes_(inverse).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# SETUP
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import time
import math
import itertools
import os
import logging
from numpy import loadtxt
import matplotlib.pyplot as plt
from keras.constraints import Constraint
import keras.backend as K

# set numpy decimal
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

# number pi
pi = math.pi

# STOP WARNING
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
logging.getLogger('tensorflow').setLevel(logging.FATAL)

# random_seed
np.random.seed(1)
tf.random.set_seed(1)

In [ ]:
# DATA
# load the dataset
with open('/content/drive/My Drive/Colab Notebooks/Keras/Solving PDEs/3. Stokes (Inverse)/output_data (Neural).csv', 'r', encoding='utf-8-sig') as f: 
    data = np.genfromtxt(f, dtype=float, delimiter=',')

# convert numpy array, P_in and P_border to Tensorflow Dataset
xy = tf.convert_to_tensor(data[:, 0:2], dtype=tf.float32)
u_v_p = tf.convert_to_tensor(data[:, 2:5], dtype=tf.float32)

dat = tf.data.Dataset.from_tensor_slices((xy, u_v_p))

batch_size = 1
bs = int(len(data))
# bs = 1

dat = dat.shuffle(buffer_size=bs).batch(batch_size)

In [ ]:
# for e in dat:
#     print(e[0].numpy()[0], e[1].numpy()[0])

In [ ]:
print("number of data: ", dat.cardinality().numpy()*batch_size)

number of data:  5625


In [ ]:
# Set condition for weight 'nu'
class Between(Constraint):
    def __init__(self, min_value, max_value):
        self.min_value = min_value
        self.max_value = max_value

    def __call__(self, nu):        
        return K.clip(nu, self.min_value, self.max_value)

    def get_config(self):
        return {'min_value': self.min_value, 'max_value': self.max_value}

In [ ]:
# MODEL
nn = 64
ini = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=1)

inputs = keras.Input(shape=(2,), name='points')
hidden_1 = layers.Dense(nn, activation='tanh', kernel_initializer=ini, name='hidden_1')(inputs)
hidden_2 = layers.Dense(nn, activation='tanh', kernel_initializer=ini, name='hidden_2')(hidden_1)
hidden_3 = layers.Dense(nn, activation='tanh', kernel_initializer=ini, name='hidden_3')(hidden_2)
# hidden_4 = layers.Dense(nn, activation='tanh', kernel_initializer=ini, name='hidden_4')(hidden_3)
# hidden_5 = layers.Dense(nn, activation='tanh', kernel_initializer=ini, name='hidden_5')(hidden_4)
# hidden_6 = layers.Dense(nn, activation='tanh', kernel_initializer=ini, name='hidden_6')(hidden_5)
# hidden_7 = layers.Dense(nn, activation='tanh', kernel_initializer=ini, name='hidden_7')(hidden_6)
# hidden_8 = layers.Dense(nn, activation='tanh', kernel_initializer=ini, name='hidden_8')(hidden_7)
# hidden_9 = layers.Dense(nn, activation='tanh', kernel_initializer=ini, name='hidden_9')(hidden_8)
outputs = layers.Dense(3, activation='linear', kernel_initializer=ini, name="u_v_p")(hidden_3)
model = keras.Model(inputs=inputs, outputs=outputs, name='stokes')
model.add_weight(name="nu", shape=(1,1), dtype=tf.float32, trainable=True, initializer=ini, constraint=Between(0, 100))
model.summary()

Model: "stokes"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
points (InputLayer)          [(None, 2)]               0         
_________________________________________________________________
hidden_1 (Dense)             (None, 64)                192       
_________________________________________________________________
hidden_2 (Dense)             (None, 64)                4160      
_________________________________________________________________
hidden_3 (Dense)             (None, 64)                4160      
_________________________________________________________________
u_v_p (Dense)                (None, 3)                 195       
Total params: 8,708
Trainable params: 8,708
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.weights[len(model.weights) - 1]

<tf.Variable 'nu:0' shape=(1, 1) dtype=float32, numpy=array([[0.006]], dtype=float32)>

In [ ]:
# CHOOSE OPTIMIZER
optimizer = tf.keras.optimizers.Adam()

In [ ]:
# TRAIN FUNCTION
@tf.function
def train_step(dat_batch):
    P = dat_batch[0]
    uvp = dat_batch[1]

    with tf.GradientTape(persistent=True) as tape:
        batch_loss = 0

        # loss
        for i in range(P.shape[0]):
            z = tf.reshape(P[i], shape=(1,2))
            tape.watch(z)
            model_vals = model(z)

            # calculating u_xx, u_yy, v_xx, v_yy, p_x, p_y
            u = model_vals[0][0]
            v = model_vals[0][1]
            p = model_vals[0][2]

            u_x = tape.gradient(u, z)[0][0]
            u_xx = tape.gradient(u_x, z)[0][0]
            u_y = tape.gradient(u, z)[0][1]
            u_yy = tape.gradient(u_y, z)[0][1]
            
            v_x = tape.gradient(v, z)[0][0]
            v_xx = tape.gradient(v_x, z)[0][0]
            v_y = tape.gradient(v, z)[0][1]
            v_yy = tape.gradient(v_y, z)[0][1]

            p_x = tape.gradient(p, z)[0][0]
            p_y = tape.gradient(p, z)[0][1]

            # calculating loss
            nu = model.weights[len(model.weights) - 1]
            loss1 = tf.math.square(nu*(u_xx + u_yy) - p_x)
            loss2 = tf.math.square(nu*(v_xx + v_yy) - p_y)
            loss3 = tf.math.square(u_x + v_y)
            loss4 = tf.math.square(u - uvp[0][0]) + tf.math.square(v - uvp[0][1]) + tf.math.square(p - uvp[0][2])
            batch_loss += loss1 + loss2 + loss3 + loss4
    
    # update weights
    grads = tape.gradient(batch_loss, model.trainable_weights)          # gradient of loss function with respect to w
    optimizer.apply_gradients(zip(grads, model.trainable_weights))      # update w

    return batch_loss

In [ ]:
# TRAIN
losses = []
def train(epochs, learning_rate):
    start_time = time.time()
    optimizer.learning_rate.assign(learning_rate)
    for epoch in range(epochs+1):
        loss = 0
        for dat_batch in dat:
            batch_loss = train_step(dat_batch)
            loss += batch_loss
        losses.append(loss.numpy())

        if(epoch%5 == 0):
            print("[%3s] Training loss: %15.3f \t nu = %.5f" % (epoch, float(loss), model.weights[len(model.weights) - 1]))

    print("Batch time: %.2f minutes" % ((time.time() - start_time)/60))

In [ ]:
lr = 1e-3
ite = 3
start_time = time.time()
for i in range(ite):
    print("Iteration:", i, "\t lr:", lr)
    train(200, lr)
    lr = lr/10
    print()
print("Total time: %.5f minutes" % ((time.time() - start_time)/60))

Iteration: 0 	 lr: 0.001
[  0] Training loss:       22354.092 	 nu = 2.22902
[  5] Training loss:        2372.718 	 nu = 2.20956
[ 10] Training loss:        1655.268 	 nu = 2.21599
[ 15] Training loss:        1240.931 	 nu = 2.19952
[ 20] Training loss:        1103.377 	 nu = 2.11379
[ 25] Training loss:         876.074 	 nu = 1.99950
[ 30] Training loss:         810.255 	 nu = 1.87947
[ 35] Training loss:         780.255 	 nu = 1.76284
[ 40] Training loss:         779.626 	 nu = 1.64233
[ 45] Training loss:         670.041 	 nu = 1.53885
[ 50] Training loss:         639.197 	 nu = 1.44285
[ 55] Training loss:         631.475 	 nu = 1.35991
[ 60] Training loss:         551.741 	 nu = 1.28777
[ 65] Training loss:         520.187 	 nu = 1.22093
[ 70] Training loss:         534.075 	 nu = 1.16735
[ 75] Training loss:         486.216 	 nu = 1.12585
[ 80] Training loss:         449.208 	 nu = 1.09618
[ 85] Training loss:         451.854 	 nu = 1.06933
[ 90] Training loss:         470.865 	 

In [ ]:
z = tf.convert_to_tensor([[0.5, 0.0]])
print("u(",z.numpy()[0][0], z.numpy()[0][1],") \t = %s" % model(z).numpy()[0][0:2])
z = tf.convert_to_tensor([[0.5, 0.1]])
print("u(",z.numpy()[0][0], z.numpy()[0][1],") \t = %s" % model(z).numpy()[0][0:2])
z = tf.convert_to_tensor([[0.5, 0.2]])
print("u(",z.numpy()[0][0], z.numpy()[0][1],") \t = %s" % model(z).numpy()[0][0:2])
z = tf.convert_to_tensor([[0.5, 0.3]])
print("u(",z.numpy()[0][0], z.numpy()[0][1],") \t = %s" % model(z).numpy()[0][0:2])
z = tf.convert_to_tensor([[0.5, 0.4]])
print("u(",z.numpy()[0][0], z.numpy()[0][1],") \t = %s" % model(z).numpy()[0][0:2])
z = tf.convert_to_tensor([[0.5, 0.5]])
print("u(",z.numpy()[0][0], z.numpy()[0][1],") \t = %s" % model(z).numpy()[0][0:2])
z = tf.convert_to_tensor([[0.5, 0.6]])
print("u(",z.numpy()[0][0], z.numpy()[0][1],") \t = %s" % model(z).numpy()[0][0:2])
z = tf.convert_to_tensor([[0.5, 0.7]])
print("u(",z.numpy()[0][0], z.numpy()[0][1],") \t = %s" % model(z).numpy()[0][0:2])
z = tf.convert_to_tensor([[0.5, 0.8]])
print("u(",z.numpy()[0][0], z.numpy()[0][1],") \t = %s" % model(z).numpy()[0][0:2])
z = tf.convert_to_tensor([[0.5, 0.9]])
print("u(",z.numpy()[0][0], z.numpy()[0][1],") \t = %s" % model(z).numpy()[0][0:2])
z = tf.convert_to_tensor([[0.5, 1.0]])
print("u(",z.numpy()[0][0], z.numpy()[0][1],") \t = %s" % model(z).numpy()[0][0:2])

u( 0.5 0.0 ) 	 = [-0.015 0.005]
u( 0.5 0.1 ) 	 = [-0.045 0.003]
u( 0.5 0.2 ) 	 = [-0.083 0.004]
u( 0.5 0.3 ) 	 = [-0.121 0.004]
u( 0.5 0.4 ) 	 = [-0.149 0.005]
u( 0.5 0.5 ) 	 = [-0.151 0.004]
u( 0.5 0.6 ) 	 = [-0.108 0.002]
u( 0.5 0.7 ) 	 = [0.006 0.000]
u( 0.5 0.8 ) 	 = [0.219 -0.001]
u( 0.5 0.9 ) 	 = [0.559 -0.002]
u( 0.5 1.0 ) 	 = [1.046 -0.004]


In [ ]:
model.save("/content/drive/My Drive/Colab Notebooks/Keras/Solving PDEs/3. Stokes (Inverse)")

In [ ]:
model1 = keras.models.load_model("/content/drive/My Drive/Colab Notebooks/Keras/Solving PDEs/3. Stokes (Inverse)")